# FAIR workbench demo using `FairStep.execute()` method.
In this notebook we demo using the `FairStep.execute()` method for executing the raw code in the `description` of the step.

We provide a working example of the happy flow, but also selection of examples showing that a `.execute` might be quite a bad idea.

We provide short code examples, but you have to imagine that the steps are published as RDF, then fetched again and run on some other machine.

## Advantages:
* Simple and easy-to-use (at least for the happy flow)

## Disadvantages:
* Highly irreproducible, since we only store the code and not all dependencies & state(i.e. python libraries, environment)
* Insecure, since we can execute malicious code

In [42]:
import inspect
from fairworkflows import FairStep, FairWorkflow

## Introducing the `.execute` method
Calling `FairStep.execute` calls the function that is stored in the description of the step,
thereby successfully executing the step.

This is a short demonstration, but you have to imagine that this step is published as RDF, then fetched again and run on some other machine.

In [27]:
def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return a + b
# Here we add the code as the description of the step. NB: this can be made more user-friendly
step = FairStep(description=inspect.getsource(add))  

In [28]:
step.execute(40, 2)

42

## Examples of problems for a `.execute` method

### Executing malicious code
The code in the description of a fair step could be malicious, for example deleting all your files.

In [13]:
def you_can_trust_me(a: int, b: int) -> int:
    """
    Trust me, you can trust me!
    """
    print('Deleting all your files')
step = FairStep(description=inspect.getsource(you_can_trust_me))

In [14]:
step.execute(40, 2)

Deleting all your files


### Execution depending on state
This code adds a constant `TWO` to the input. The constant is however not defined in the code.

This also demonstrates the case where the code depends on an environment variable.

In [35]:
TWO = 2
def add_constant(a: int) -> int:
    """
    Computational step adding 2 to the input.
    """
    return a + TWO
step = FairStep(description=inspect.getsource(add_constant))

In [34]:
step.execute(40)

NameError: name 'TWO' is not defined

### Execution depending on imported packages
This function depends on the package `package_i_do_not_have`

In [39]:
def add_unknown_import(a: int, b: int) -> int:
    """
    Computational step adding two ints together using an unknown package.
    """
    import package_i_do_not_have
    return package_i_do_not_have.sum([a, b])
step = FairStep(description=inspect.getsource(add_unknown_import))

In [38]:
step.execute(40, 2)

ModuleNotFoundError: No module named 'package_i_do_not_have'

### Execution of nested function
Here the code of `add` depends on the function `_add` that was not included in the step description.

In theory we could include all code that is being called inside the function, but this can end up including whole programs in the RDF description.

In [40]:
def _add(a: int, b: int) -> int:
    return a + b

def add(a: int, b: int) -> int:
    """
    Computational step adding two ints together.
    """
    return _add(a + b)
step = FairStep(description=inspect.getsource(add))  

In [41]:
step.execute(40, 2)

NameError: name '_add' is not defined